## Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice. 



In [1]:
!python -V

Python 3.10.13


In [35]:
import json
from pprint import pprint
from tqdm.notebook import tqdm

## Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?




In [3]:
ELASTIC_SEARCH_INFO = 'elastic-search-info.json'

In [4]:
!curl localhost:9200  -o {ELASTIC_SEARCH_INFO}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 9200: Connection refused


In [5]:


with open(ELASTIC_SEARCH_INFO) as json_data:
    info = json.load(json_data)
    json_data.close()
    pprint(info)

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'd4Tl-ZpEQ1aXu6j70MNdww',
 'name': '63f9977d5bd6',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2022-10-04T07:17:24.662462378Z',
             'build_flavor': 'default',
             'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.3.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.4.3'}}


In [6]:
info['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```


In [12]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```



https://elasticsearch-py.readthedocs.io/en/v8.14.0/



In [8]:
!pip install elasticsearch

In [9]:
!conda install elasticsearch -y

Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [39]:
from datetime import datetime
from elasticsearch import Elasticsearch

#del client

In [40]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

{'name': '6168a5eaed52',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'G9MnYmkNRIei86p2u1V5-Q',
 'version': {'number': '8.4.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73',
  'build_date': '2022-10-04T07:17:24.662462378Z',
  'build_snapshot': False,
  'lucene_version': '9.3.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [41]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-question"

In [42]:
# Obtener lista de índices y imprimir sus nombres
try:
    indices = es_client.cat.indices(format='json')
    for index in indices:
        print(index['index'])
except Exception as e:
    print(f'Error al obtener los índices: {e}')


In [44]:
#es_client.indices.delete(index=index_name)

In [45]:
es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-question'}

In [46]:
for doc in tqdm(documents):
    #print(doc)
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Which function do you use for adding your data to elastic?

* `insert`
* **`index`**
* `put`
* `add`

In [28]:
#!cat /opt/conda/envs/intro-env/lib/python3.10/site-packages/elasticsearch/client/utils.py

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.



In [60]:
query = 'How do I execute a command in a running docker container?'

In [61]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [62]:
response = es_client.search(index=index_name, body=search_query)

In [63]:
response

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 390, 'relation': 'eq'},
  'max_score': 75.54128,
  'hits': [{'_index': 'course-question',
    '_id': 'oMqiO5AB5e7Yp_BPDL2A',
    '_score': 75.54128,
    '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n

In [69]:
response['hits']['max_score']

75.54128

In [70]:
response['hits']['hits'][0]['_score']

75.54128

What's the score for the top ranking result?

* 94.05
* 84.05
* **74.05**
* 64.05

Look at the `_score` field.



## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```
